In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import matplotlib.pyplot as plt
import math
import cv2

In [ ]:
gray_lena_img = cv2.imread('/content/drive/MyDrive/Colab Notebooks/cv_2024_1/lena.bmp', cv2.IMREAD_GRAYSCALE) #이미지를 흑백이미지로 읽음

In [ ]:
# 1. Design 𝑘 × 𝑘 Box filter generation function.
# Input : filter size (𝑘)
# Output : 𝑘 ×𝑘 filter kernel

def box_filter(k):
    return np.ones((k, k)) / (k * k) # box filter 공식

box_filter_mask = box_filter(5)


In [ ]:
#1 번 교수님버전
def box_filter(k):
    f=np.ones((k,k))
    f/=np.sum(k*k)
    return f

b_filter=box_filter(3)
print(b_filter)
print(np.sum(b_filter))

In [ ]:
# 2. Design image filtering (convolution) function, and apply it to the noisy image for denoising.
# Input : image (𝑀 × 𝑁), filter (𝑘 × 𝑘)
# Output : filtered image (𝑀 × 𝑁)

# 가우시안 노이즈 추가 함수: mean=0,sigma^2인 가우시안 노이즈를 이미지에 추가
def add_gaussian_noise(image, sigma):
    row, col = image.shape
    # uniform_noise = (np.random.random((row, col))-0.5)*100 # uniform 노이즈 추가
    mean=0
    gaussian_noise = np.random.normal(mean,sigma,(row,col)) # gaussian 노이즈 추가
    noisy_image = image + gaussian_noise
    noisy_image = np.clip(noisy_image, 0, 255) #화솟값 제한(0~255)
    return noisy_image.astype(np.uint8)

def convolution(image, filter):

  image_height, image_width = image.shape
  filter_height, filter_width = filter.shape
  padding = filter_height // 2
  filtered_image = np.zeros_like(image) #새로운 복사 이미지

  #반복문을 수행하면서 패딩&컨볼루션 진행
  for y in range(padding, image_height - padding):
    for x in range(padding, image_width - padding):
      image_patch = image[y - padding:y + padding + 1, x - padding:x + padding + 1]
      filtered_image[y, x] = np.sum(image_patch * filter)

  return filtered_image

#이미지에 가우시안 노이즈 추가
gaussian_noise_lena_img=add_gaussian_noise(gray_lena_img, 50)

# 가우시안 노이즈 이미지에 박스필터 적용
box_filtered_lena_img = convolution(gaussian_noise_lena_img, box_filter_mask)

# 가우시안 노이즈 이미지와 박스필터 적용 이미지 비교
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(gaussian_noise_lena_img, cmap='gray')
plt.title('gaussian noise image')

plt.subplot(1, 2, 2)
plt.imshow(box_filtered_lena_img, cmap='gray')
plt.title('Filtered image (Box Filter 5x5)')
plt.show()


In [ ]:
#2번 교수님 버전

# 교수님 버전 box filter 함수
def box_filter(k):
    f=np.ones((k,k))
    f/=np.sum(k*k)
    return f


# 교수님 버전 convolution진행 함수
def filter2D(img, f):
    out_img=img.copy() # 이미지 복사
    k = f.shape[0]
    k_half = k//2
    height, weight=img.shape

    for y in range(k_half, height-k_half):
        for x in range(k_half, weight-k_half):
            img_block=img[y-k_half : y+k_half+1, x-k_half : x+k_half+1] # k x k 으로 진행
            out_img[y,x]=np.sum(img_block*f)
    return out_img

f = box_filter(5)
f_img=filter2D(gaussian_noise_lena_img,f)

# 가우시안 노이즈 이미지와 박스필터 적용 이미지 비교
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(gaussian_noise_lena_img, cmap='gray')
plt.title('gaussian noise image')

plt.subplot(1, 2, 2)
plt.imshow(f_img, cmap='gray')
plt.title('Filtered image (Box Filter 5x5)')
plt.show()

In [ ]:
# 2번 내꺼와 교수님 버전 psnr 비교
def psnr_func(image1,image2): # 2개의 이미지의 크기기 같아야 됨
    mse=np.mean((image1-image2)**2) #mse 계산식
    psnr=20*math.log10(255/math.sqrt(mse)) #psnr 계산식
    return mse, psnr

val1,val2=psnr_func(gaussian_noise_lena_img, gaussian_filtered_lena_img)

print("MSE : ",val1," PSNR : ",val2)



In [ ]:
# 3. Design 𝑘 × 𝑘 Gaussian filter generation function.
# Input : Gaussian parameter, i.e., standard deviation (std) for Gaussian kernel
# Output : 𝑘 ×𝑘 filter kernel

def gaussian_filter(k,sig):
    ax = np.linspace(-(k//2), k//2, k)
    x, y = np.meshgrid(ax, ax)
    kernel = np.exp(-(x**2 + y**2) / (2 * sig**2)) # k x k 크기의 커널 생성
    return kernel / np.sum(kernel) # 정규화. 전체 필터의 요소들의 합은 1

gaussian_filter_mask = gaussian_filter(5,1.5)
print(gaussian_filter_mask)
print(np.sum(gaussian_filter_mask))

In [ ]:
# 4. Repeat #2 using Gaussian filter

# 가우시안 노이즈 이미지에 가우시안 필터 적용
gaussian_filtered_lena_img = convolution(gaussian_noise_lena_img, gaussian_filter_mask)

# 가우시안 노이즈 이미지와 가우시안 필터 적용 이미지 비교
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.imshow(gaussian_noise_lena_img, cmap='gray')
plt.title('gaussian noise image')

plt.subplot(1, 3, 2)
plt.imshow(box_filtered_lena_img, cmap='gray')
plt.title('Filtered image (Gaussian Filter 5x5)')

plt.subplot(1,3,3)
plt.imshow(gray_lena_img, cmap='gray')
plt.title('Original img')
plt.show()


📌 PSNR 해석
PSNR 값이 높을수록 품질이 좋고, 원본과 유사함.

PSNR 값이 낮을수록 품질이 나쁘고, 원본과 차이가 큼.

PSNR (dB)	품질 해석
40 dB 이상	매우 좋음 (원본과 거의 동일)
30~40 dB	양호 (약간의 손실)
20~30 dB	보통 (노이즈 또는 품질 저하가 보임)
20 dB 이하	품질이 낮음


📌 원본 vs 노이즈 추가 vs 필터링된 이미지의 PSNR 비교
1. 원본 이미지 vs 노이즈가 추가된 이미지
노이즈가 추가되면 픽셀 값이 변하므로 MSE 증가 → PSNR 감소

PSNR 값이 낮고, 심한 노이즈일 경우 20dB 이하일 수도 있음.

2. 원본 이미지 vs 필터링된 이미지
필터링하면 노이즈가 줄어들지만, 엣지 손실이나 블러 효과가 생길 수 있음.

잘 필터링되면 PSNR이 증가하지만, 너무 강한 필터링은 원본과 차이를 만들어 PSNR을 낮출 수도 있음.

3. 노이즈가 추가된 이미지 vs 필터링된 이미지
필터링 후에는 노이즈가 줄어들어 MSE 감소 → PSNR 증가

PSNR 값이 원본과 비교한 것보다는 높게 나올 가능성이 큼.

In [ ]:
# gpt psnr 예시 코드

def compute_psnr(original, modified):
    mse = np.mean((original - modified) ** 2)
    if mse == 0:
        return float('inf')  # 두 이미지가 완전히 동일하면 PSNR은 무한대!!!
    max_pixel = 255.0
    psnr = 10 * np.log10((max_pixel ** 2) / mse)
    return psnr

In [ ]:
# 5. Measure the MSE and PSNR for noisy and filtered images.
# psnr은 항상 원본-노이즈된 영상 / 원본-필터링된 연산 이런식으로 비교
# PSNR 값이 높을수록 품질이 좋고, 원본과 유사함.
# PSNR 값이 낮을수록 품질이 나쁘고, 원본과 차이가 큼.
'''
PSNR 값의 범위 (dB)
0 dB 이상	매우 좋음 (원본과 거의 동일)
30~40 dB	양호 (약간의 손실)
20~30 dB	보통 (노이즈 또는 품질 저하가 보임)
20 dB 이하	품질이 낮음
'''

# 가우시안 노이즈 이미지와 가우시안 필터 이미지의 mse & psnr 계산
def psnr_func(image1,image2): # 2개의 이미지의 크기기 같아야 됨
    mse=np.mean((image1-image2)**2) #mse 계산식
    psnr=20*math.log10(255/math.sqrt(mse)) #psnr 계산식
    return mse, psnr

val1,val2=psnr_func(gray_lena_img, gaussian_filtered_lena_img)

print("MSE : ",val1," PSNR : ",val2)


In [ ]:
# 추가 실습---bilateral filter 구현
def bilateral_filter(image, k, sigma_r, sigma_s):
    h, w = image.shape
    output = np.zeros_like(image, dtype=np.float32)

    # 필터 크기 설정
    radius = k // 2

    # 가우시안 공간 필터 생성
    spatial_gaussian = np.zeros((k, k), dtype=np.float32)
    for i in range(k):
        for j in range(k):
            x, y = i - radius, j - radius
            spatial_gaussian[i, j] = np.exp(-(x**2 + y**2) / (2 * sigma_s**2))

    # 바이레터럴 필터 적용
    for y in range(radius, h - radius):
        for x in range(radius, w - radius):
            region = image[y - radius:y + radius + 1, x - radius:x + radius + 1]

            # 색 강도 가우시안 필터 생성
            intensity_gaussian = np.exp(-((region - image[y, x])**2) / (2 * sigma_r**2))

            # 최종 가중치
            weights = spatial_gaussian * intensity_gaussian

            # 가중 평균 계산
            output[y, x] = np.sum(weights * region) / np.sum(weights)

    return np.uint8(output)

bilateral_filtered_img = bilateral_filter(gaussian_noise_lena_img, k=5, sigma_r=50, sigma_s=10)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(gaussian_noise_lena_img, cmap='gray')
plt.title('gaussian noise image')

plt.subplot(1, 2, 2)
plt.imshow(bilateral_filtered_img, cmap='gray')
plt.title('Filtered image (Bilateral Filter 5x5)')
plt.show()
